In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import os
import json

In [2]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [3]:
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_bbidding1.loc[:,"market_price_lag96h"] = df_bbidding1["market_price"].shift(192)
df_bbidding1.loc[:,"imbalance_price_lag96h"] = df_bbidding1["imbalance_price"].shift(192)
df_bbidding1.loc[:,"day_ahead_price_lag1week"] = df_bbidding1["day_ahead_price"].shift(336)
df_bbidding1.loc[:,"volume_lag96h"] = df_bbidding1["volume"].shift(192)
df_bbidding1.dropna(inplace=True)
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h
336,2021-03-04 08:00:00+00:00,68.55,73.77,1201.25,17,-0.500000,0.467359,106.202828,126.692556,188.827864,...,248.421344,278.463233,323.809394,373.852140,463.068992,89.95,7.54,8.00,32.38,755.80
337,2021-03-04 08:30:00+00:00,69.25,73.77,1105.35,18,-0.500000,0.467359,109.169687,146.438753,203.691171,...,237.873544,290.936786,336.975817,382.262617,516.284064,89.00,-9.71,7.00,32.38,900.35
338,2021-03-04 09:00:00+00:00,75.18,70.86,1245.05,19,-0.707107,0.467359,85.628752,156.903984,199.220095,...,238.966779,300.425701,345.177645,375.303556,488.195210,92.00,55.58,33.65,17.34,497.60
339,2021-03-04 09:30:00+00:00,73.19,70.86,1590.00,20,-0.707107,0.467359,89.618646,163.000319,205.983800,...,263.775945,307.808096,356.190088,404.786859,445.870108,92.00,57.95,62.70,17.34,554.50
340,2021-03-04 10:00:00+00:00,74.12,63.72,1415.30,21,-0.866025,0.467359,140.104118,259.247292,313.074122,...,350.917349,469.297198,491.961995,584.346472,624.029818,98.00,53.11,34.10,14.25,739.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,28.044030,85.584636,147.450968,...,257.545157,312.000836,389.209655,461.471346,565.748439,66.01,91.88,126.00,50.21,1271.20
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-2.130944,44.062739,80.500728,...,152.946134,213.340381,285.856261,352.111989,494.824570,66.01,90.64,90.64,37.82,1643.15
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-53.850807,-31.030380,1.674412,...,80.555516,138.624007,177.132223,250.514362,333.223882,66.01,95.15,115.99,37.82,1599.25
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-42.706652,-38.988480,19.495393,...,72.199724,128.661011,157.102188,214.652927,345.416021,66.01,91.30,125.00,31.66,1619.70


In [4]:
# Original dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
y = df_bbidding1["imbalance_price"].values  # Convert to numpy

# Step 1: Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 2: Further split the training set into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Step 3: Standardize the data (use only training data to fit the scaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
#save the scaler
pickle.dump(scaler, open("LSTM_imbalance_scaler.pkl", "wb"))

In [5]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [6]:
# y_train = y_train.values
# y_test = y_test.values

In [7]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1                  # Dropout rate
learning_rate = 0.0001         # Learning rate for optimizer
batch_size = 32                # Batch size
num_epochs = 500               # Maximum number of epochs
patience = 15                  # Patience for early stopping
rel_improvement_threshold = 0.0001  # Relative improvement threshold

In [8]:
input_size

15

In [9]:
hidden_size

64

In [10]:
model = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout)

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_loss = float('inf')
best_model = None
patience_counter = 0
previous_loss = float('inf')

In [11]:
import torch.nn.functional as F  # For MAE loss function

best_val_loss = float('inf')  # Initialize the best validation loss
best_model = None
patience_counter = 0  # For early stopping

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0

    # Training loop
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension (for some models)
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the loss using MAE
        loss = F.l1_loss(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Reshape input to add sequence length dimension
            X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

            # Forward pass
            y_pred = model(X_batch)  # (batch_size, output_size)

            # Compute the validation loss using MAE
            val_loss = F.l1_loss(y_pred, y_batch)
            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

    # Early stopping based on validation loss
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        best_model = model.state_dict()  # Save the best model's weights
        patience_counter = 0  # Reset patience counter
        print(f"Best model saved with validation loss: {best_val_loss:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
if best_model is not None:
    model.load_state_dict(best_model)
else:
    print("No improvement was observed during training.")

torch.save(model.state_dict(), 'LSTM_imbalance_price.pth')

# Test the model using the best model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the test loss using MAE
        loss = F.l1_loss(y_pred, y_batch)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')


Epoch [1/500], Training Loss: 161.6005
Epoch [1/500], Validation Loss: 149.7111
Best model saved with validation loss: 149.7111
Epoch [2/500], Training Loss: 152.9179
Epoch [2/500], Validation Loss: 144.7974
Best model saved with validation loss: 144.7974
Epoch [3/500], Training Loss: 148.3845
Epoch [3/500], Validation Loss: 141.0752
Best model saved with validation loss: 141.0752
Epoch [4/500], Training Loss: 144.4170
Epoch [4/500], Validation Loss: 137.6364
Best model saved with validation loss: 137.6364
Epoch [5/500], Training Loss: 140.6555
Epoch [5/500], Validation Loss: 134.3360
Best model saved with validation loss: 134.3360
Epoch [6/500], Training Loss: 137.0239
Epoch [6/500], Validation Loss: 131.1920
Best model saved with validation loss: 131.1920
Epoch [7/500], Training Loss: 133.5244
Epoch [7/500], Validation Loss: 128.1892
Best model saved with validation loss: 128.1892
Epoch [8/500], Training Loss: 130.1087
Epoch [8/500], Validation Loss: 125.2668
Best model saved with va

In [12]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [13]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new

,prediction_date,timestamp,1,2,3,4,5,6,7,8,9
0,2024-10-04,2024-10-03T22:00:00+00:00,1355,487,963,1544,1330,867,326,1402,1357
1,2024-10-04,2024-10-03T22:30:00+00:00,788,584,1148,1320,1007,565,923,797,487
2,2024-10-04,2024-10-03T23:00:00+00:00,571,748,1001,1460,740,1017,1533,597,991
3,2024-10-04,2024-10-03T23:30:00+00:00,1476,1194,1512,334,1054,1572,744,1497,463
4,2024-10-04,2024-10-04T00:00:00+00:00,1352,1390,912,662,1505,1093,814,487,1340
...,...,...,...,...,...,...,...,...,...,...,...
811,2024-10-19,2024-10-19T19:30:00+00:00,3,7,23,32,45,57,77,93,125
812,2024-10-19,2024-10-19T20:00:00+00:00,0,11,25,36,42,60,82,103,132
813,2024-10-19,2024-10-19T20:30:00+00:00,0,16,27,39,44,64,89,106,140
814,2024-10-19,2024-10-19T21:00:00+00:00,0,11,29,42,51,75,97,113,151


In [14]:
df_api_new["timestamp"] = pd.to_datetime(df_api_new["timestamp"])
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.day/7)
df_api_new_merged

,prediction_date,timestamp,1,2,3,4,5,6,7,8,...,price_y,volume,timestamp_utc,settlement_date,settlement_period,imbalance_price,day_ahead_price,market_price,cos_hour,cos_day
0,2024-10-04,2024-10-03 22:00:00+00:00,1355,487,963,1544,1330,867,326,1402,...,71.32,1322.45,2024-10-03 22:00:00+00:00,2024-10-03,47.0,61.0,78.47,71.32,0.866025,-0.900969
1,2024-10-04,2024-10-03 22:30:00+00:00,788,584,1148,1320,1007,565,923,797,...,71.42,1035.65,2024-10-03 22:30:00+00:00,2024-10-03,48.0,50.0,78.47,71.42,0.866025,-0.900969
2,2024-10-04,2024-10-03 23:00:00+00:00,571,748,1001,1460,740,1017,1533,597,...,66.94,1378.55,2024-10-03 23:00:00+00:00,2024-10-04,1.0,93.5,71.36,66.94,0.965926,-0.900969
3,2024-10-04,2024-10-03 23:30:00+00:00,1476,1194,1512,334,1054,1572,744,1497,...,60.03,1539.90,2024-10-03 23:30:00+00:00,2024-10-04,2.0,93.5,71.36,60.03,0.965926,-0.900969
4,2024-10-04,2024-10-04 00:00:00+00:00,1352,1390,912,662,1505,1093,814,487,...,65.21,1818.40,2024-10-04 00:00:00+00:00,2024-10-04,3.0,93.0,72.80,65.21,1.000000,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818,2024-10-19,2024-10-19 19:30:00+00:00,3,7,23,32,45,57,77,93,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.258819,-0.222521
819,2024-10-19,2024-10-19 20:00:00+00:00,0,11,25,36,42,60,82,103,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,-0.222521
820,2024-10-19,2024-10-19 20:30:00+00:00,0,16,27,39,44,64,89,106,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,-0.222521
821,2024-10-19,2024-10-19 21:00:00+00:00,0,11,29,42,51,75,97,113,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.707107,-0.222521


In [15]:
df_api_new_merged1 = df_api_new_merged[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(336)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(192)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h
336,2024-10-13 18:30:00+00:00,0.00,101.24,0.00,40.0,-1.836970e-16,0.62349,960,1020,1081,...,1201,1261,1322,1382,1443,159.00,61.97,94.00,78.47,2851.35
337,2024-10-13 19:00:00+00:00,0.00,89.62,0.00,41.0,2.588190e-01,0.62349,960,1021,1081,...,1201,1261,1321,1381,1441,159.00,55.09,55.09,78.47,2360.40
338,2024-10-13 19:30:00+00:00,0.00,89.62,0.00,42.0,2.588190e-01,0.62349,960,1021,1081,...,1201,1261,1321,1381,1441,159.00,74.08,93.45,71.36,861.85
339,2024-10-13 20:00:00+00:00,0.00,84.30,0.00,43.0,5.000000e-01,0.62349,0,0,0,...,0,0,0,0,0,70.00,66.77,90.00,71.36,1152.85
340,2024-10-13 20:30:00+00:00,0.00,84.30,0.00,44.0,5.000000e-01,0.62349,0,0,0,...,0,0,0,0,0,70.00,47.21,90.00,72.80,1095.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,2.588190e-01,0.62349,850,921,980,...,1060,1081,1104,1123,1142,105.00,0.00,159.00,82.15,0.00
531,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,5.000000e-01,0.62349,897,964,1010,...,1074,1097,1111,1129,1151,104.90,0.00,70.00,79.14,0.00
532,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,5.000000e-01,0.62349,910,984,1040,...,1101,1120,1122,1139,1154,64.00,0.00,70.00,79.14,0.00
533,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,7.071068e-01,0.62349,938,1034,1069,...,1124,1135,1139,1152,1164,33.77,0.00,129.00,91.45,0.00


In [16]:
df_api_new_merged1 = df_api_new_merged1.groupby("timestamp_utc").last().reset_index()

In [17]:
X_test = df_api_new_merged1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X_test = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
X_test_tensor.shape

torch.Size([103, 15])

In [18]:
X_test_tensor = X_test_tensor.unsqueeze(1)  # Adds a sequence length dimension


In [19]:


model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged1["imbalance_price_predictions"] = final_test_outputs
df_api_new_merged1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h,imbalance_price_predictions
0,2024-10-13 18:30:00+00:00,0.00,101.24,0.00,40.0,-1.836970e-16,0.62349,960,1020,1081,...,1261,1322,1382,1443,159.00,61.97,94.00,78.47,2851.35,66.798347
1,2024-10-13 19:00:00+00:00,0.00,89.62,0.00,41.0,2.588190e-01,0.62349,960,1021,1081,...,1261,1321,1381,1441,159.00,55.09,55.09,78.47,2360.40,68.725052
2,2024-10-13 19:30:00+00:00,0.00,89.62,0.00,42.0,2.588190e-01,0.62349,960,1021,1081,...,1261,1321,1381,1441,159.00,74.08,93.45,71.36,861.85,91.374313
3,2024-10-13 20:00:00+00:00,0.00,84.30,0.00,43.0,5.000000e-01,0.62349,0,0,0,...,0,0,0,0,70.00,66.77,90.00,71.36,1152.85,76.828766
4,2024-10-13 20:30:00+00:00,0.00,84.30,0.00,44.0,5.000000e-01,0.62349,0,0,0,...,0,0,0,0,70.00,47.21,90.00,72.80,1095.30,70.860718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,2.588190e-01,0.62349,850,921,980,...,1081,1104,1123,1142,105.00,0.00,159.00,82.15,0.00,105.983086
99,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,5.000000e-01,0.62349,897,964,1010,...,1097,1111,1129,1151,104.90,0.00,70.00,79.14,0.00,88.226570
100,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,5.000000e-01,0.62349,910,984,1040,...,1120,1122,1139,1154,64.00,0.00,70.00,79.14,0.00,87.353539
101,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,7.071068e-01,0.62349,938,1034,1069,...,1135,1139,1152,1164,33.77,0.00,129.00,91.45,0.00,92.058922


In [20]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged1["timestamp_utc"], y=df_api_new_merged1["imbalance_price"], mode='lines', name='Imbalance Price'))
fig.add_trace(go.Scatter(x=df_api_new_merged1["timestamp_utc"], y=df_api_new_merged1["imbalance_price_predictions"], mode='lines', name='Imbalance Price Predictions'))
fig.update_layout(title='Imbalance Price and Predictions', xaxis_title='Timestamp', yaxis_title='Imbalance Price')
fig.show()

In [21]:
#MAE calculation for imbalance price predictions
mae = np.mean(np.abs(df_api_new_merged1["imbalance_price"] - df_api_new_merged1["imbalance_price_predictions"]))
print(f"MAE for imbalance price predictions: {mae:.4f}")

MAE for imbalance price predictions: 59.5189


In [22]:
#predict for the whole dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X = scaler.transform(X)
X_tensor = torch.tensor(X, dtype=torch.float32)
X_tensor = X_tensor.unsqueeze(1)  # Adds a sequence length

model.eval()
with torch.no_grad():
    final_outputs = model(X_tensor)
    final_outputs = final_outputs.numpy().flatten()

df_bbidding1["imbalance_price_predictions"] = final_outputs

In [23]:
df_bbidding1.to_csv('bidding_training_predictions_imbalance_price.csv', index=False)